# Scraping Metal Archives et Sens Critique 

# Scrapping avec Sélénium et Beautiful Soup

## Métal-Archives

In [28]:
from bs4 import BeautifulSoup
import requests 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import numpy as np
import time
import pandas as pd
import re

In [2]:
head={'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
'Accept-Encoding':'gzip, deflate, br',
'Accept-Language':'fr,fr-FR;q=0.8,en-US;q=0.5,en;q=0.3',
'Cache-Control':'max-age=0',
'Connection':'keep-alive',
'Cookie':'__cfduid=d84fa24385159719c98a949348f9a95dc1600775800; PHPSESSID=qndh4k55iict30krmqh99s1cm6; __utma=235797405.191129115.1600775802.1600849933.1600852881.3; __utmc=235797405; __utmz=235797405.1600775802.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); __cf_bm=9b057e125a8c3257f4868d739b0342f81338406d-1600852879-1800-ATvONaTDHX6GG2KSbqnZW3JQRUcplX5rO5IuRtoWO3+meCATjHOJHMN1xwd4dnURmXv/04andi3zbzKMR/NGjFs=; __utmb=235797405.35.10.1600852881; login=14812a57904b79db7d64cb216f095766356776; __utmt=1',
'Host': 'www.metal-archives.com',
'Referer': 'https://www.metal-archives.com/search?searchString=kataklysm&type=band_name',
'Upgrade-Insecure-Requests': '1',
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:80.0) Gecko/20100101 Firefox/80.0'}

band=["Arch Enemy","Kataklysm","Dagoba","Harakiri For The Sky","Cytotoxin"]

In [29]:
def scroll(driver, timeout):
    scroll_pause_time = timeout

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        
        time.sleep(scroll_pause_time)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            # If heights are the same it will exit the function
            break
        last_height = new_height


In [3]:
driver = webdriver.Firefox()
driver.get("https://www.spirit-of-metal.com/liste_groupe.php?recherche_groupe=&lettre=&id_pays_recherche=0&id_style_recherche=0&dateCrea=0&nb_etoile=2")
scroll(driver,2)

time.sleep(3)

In [4]:

groupes=driver.find_elements_by_class_name("col-xs-9")
bands=[]
for e in groupes:
    e=e.text
    e=e.split("\n")[0]
    cleanr = re.compile('(\((.*)\))')
    e= re.sub(cleanr, '',e)
    bands.append(e)
print(len(bands))

1971


In [5]:
print(bands)

['12 Stones', '12012', '1349', '25 Ta Life', '3 Inches Of Blood', '30 Seconds To Mars', '36 Crazyfists', 'A Day To Remember', 'A Different Breed Of Killer', 'A Forest Of Stars', 'A Perfect Circle', 'A Plea For Purging', 'A Skylit Drive', 'A Static Lullaby', 'A Vain Attempt', 'A9', 'Aäkon Këëtrëh', 'Aaskereia', 'Abacabb', 'Abandon All Ships', 'Abbath', 'Abigail ', 'Abigail Williams', 'Abigor', 'Abingdon Boys School', 'Abnormality ', 'Abominable Putridity', 'Aborted', 'Aborym', 'Abruptum', 'Absu', 'Abysmal Dawn', 'Abysmal Torment', 'Abyssic Hate', 'Abyssos', 'AC-DC', 'Accept', 'Accuser', 'Ace Frehley', 'Acheron ', 'Acherontas', 'Acid Bath', 'Acid Drinkers', 'Acid King', 'Acid Witch', 'Acrania ', 'Ad Hominem', 'Adagio ', 'Adelitas Way', 'Adema', 'Adept', 'Adorned Brood', 'Adrenaline Mob', 'Adrian Von Ziegler', 'ADX', 'Aeon ', 'Aerosmith', 'Aes Dana', 'Aesma Daeva', 'Aeternam', 'Aeternus', 'AFI', 'After Forever', 'After The Burial', 'Agalloch', 'Agathocles', 'Agathodaimon', 'Agent Steel', 

import pickle
# enregistrement du dictionnaire dans un fichier
Fichier = open('liste_films.txt', 'wb')
pickle.dump(bands, Fichier)    # sérialisation
Fichier.close()

In [32]:
from selenium.common.exceptions import *
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
def scrapping_spirit(band):
    try:
        driver = webdriver.Firefox()
        driver.get("https://www.spirit-of-metal.com/fr/band/{}".format(band))
        elements=driver.find_element_by_id("profile")
        element=elements.find_elements_by_tag_name("span")
        time.sleep(3)
        band_infos=[i.text for i in element]
        i=band_infos.index("Genre")
        genre=band_infos[i+1]
        i=band_infos.index("Statut")
        statut=band_infos[i+1]
        i=band_infos.index("Date de formation")
        annee=band_infos[i+1]
        i=band_infos.index("Pays")
        pays=band_infos[i+1]
        time.sleep(1.5)
        return band,pays,genre,statut,annee
    except NoSuchElementException:
        print("première méthode échouée. Nouvelle Tentative !!")
        driver=webdriver.Firefox()
        driver.get("https://www.spirit-of-metal.com/fr/bands/1")
        element=driver.find_elements_by_name("recherche_groupe")[1]
        element.clear()
        element.send_keys(band)
        driver.execute_script("window.scrollTo(1023,713);")
        recherche=driver.find_element_by_xpath("/html/body/main/div/div/aside/section[1]/div/form/input[3]")
        recherche.click()
        #driver.send_keys(Keys.ENTER)
        groupe=driver.find_elements_by_xpath("/html/body/main/div/div/div/section[3]/div[1]/div/a")
        lien=groupe[0].get_attribute("href")
        driver.get(lien)
        elements=driver.find_element_by_id("profile")
        element=elements.find_elements_by_tag_name("span")
        time.sleep(2)
        band_infos=[i.text for i in element]
        i=band_infos.index("Genre")
        genre=band_infos[i+1]
        i=band_infos.index("Statut")
        statut=band_infos[i+1]
        i=band_infos.index("Date de formation")
        annee=band_infos[i+1]
        i=band_infos.index("Pays")
        pays=band_infos[i+1]
        time.sleep(1.5)
        driver.quit()
        return band,pays,genre,statut,annee

In [33]:

def scrapping_groupes(band):
    try:
        driver = webdriver.Firefox()
        driver.get("https://www.metal-archives.com/search/advanced/?searchString=&type=")
        time.sleep(2)
        try:
            groupe2=driver.find_element_by_xpath("""//*[@id="bandName"]""")
            groupe2.clear()
            groupe2.send_keys(band)
        except (ElementNotInteractableException, StaleElementReferenceException):
            print("erreur sur la page. Page rafraîchie!!!")
            driver.refresh()
            time.sleep(3)
            #driver.execute_script("arguments[0].scrollIntoView();", groupe2)
            groupe2=driver.find_element_by_xpath("""//*[@id="bandName"]""")
            groupe2.clear()
            groupe2.send_keys(band)
            
            pass
            
            
       
        print(band)
        #genre=driver.find_element_by_id("genre")
        #genre.clear()
        #g=input("genre:")
        #genre.send_keys(g)
        bouton=driver.find_element_by_class_name("defaultButton")
        bouton.click()
        time.sleep(1)
        a=driver.find_element_by_xpath("/html/body/div/div[3]/div/div[1]/div/div/div/div/div[5]/table/tbody/tr[1]/td[1]/a").get_attribute("href")


        driver.get(a)
        print(driver.current_url)
        statut=driver.find_element_by_xpath("/html/body/div/div[3]/div[2]/div[2]/div[2]/dl[1]/dd[3]").text
        nom=driver.find_element_by_xpath("/html/body/div/div[3]/div[2]/div[2]/h1/a").text
        pays=driver.find_element_by_xpath("/html/body/div/div[3]/div[2]/div[2]/div[2]/dl[1]/dd[1]/a").text
        annees=driver.find_element_by_xpath("/html/body/div/div[3]/div[2]/div[2]/div[2]/dl[3]/dd").text
        annees= re.sub("</strong>|<dd>|</dd>", "",annees)
        cleanr = re.compile('<.*>|[(a-zA-Z)]')
        annees= re.sub(cleanr, '',annees)
        genre=driver.find_element_by_css_selector("dl.float_right > dd:nth-child(2)").text
        lyrics=driver.find_element_by_css_selector("dl.float_right > dd:nth-child(4)").text
        time.sleep(2)
        driver.quit()
        
    except (NoSuchElementException,AttributeError,TypeError,IndexError):
        print(band," non présent dans Métal Archives")
        band3=band
        return band3
    return nom,pays,statut,genre,annees,lyrics




In [8]:
gps=bands[500:]


In [11]:
import random as rd
gps[500:550]

['Knights Of The Abyss',
 'Koldbrann',
 'Koritni',
 'Korn',
 'Korpiklaani',
 'Korzus',
 'Kotipelto',
 'Kraanium',
 'Krabathor',
 'Kreator',
 'Krieg',
 'Krisiun',
 'Kristallnacht',
 'Kroda',
 'Krokus',
 'Kromlek',
 'Kronos ',
 'Krypt',
 'Krypteria',
 'Kvelertak',
 'Kvist',
 'Kylesa',
 'Kyuss',
 "L'Âme Immortelle",
 "L'Arc En Ciel",
 "L'Esprit Du Clan",
 'L7',
 'LA Guns ',
 'Laaz Rockit',
 'Labyrinth ',
 'Lacrimas Profundere',
 'Lacrimosa',
 'Lacuna Coil',
 'LaFee',
 'Laibach',
 'Lake Of Tears ',
 'Lamb Of God',
 'Lantlôs',
 'Lareine',
 'Last Days Of Humanity',
 'Lazarus AD',
 "Leaves' Eyes",
 'Led Zeppelin',
 'Lee Aaron',
 'Legion Of The Damned',
 "Leng Tch'e",
 'Leprous ',
 'Les Chants De Nihil',
 'Les Discrets',
 'Leverage']

In [12]:
rd.seed(4)
rd.shuffle(gps)

In [36]:
groupes11=gps[:50]
groupes12=gps[50:100]
groupes13=gps[100:150]
groupes13

['Eternal Tears Of Sorrow',
 'Ministry',
 'Sick Puppies',
 'Quiet Riot',
 'Theocracy',
 'Ljå',
 'Jag Panzer',
 'Upon This Dawning',
 'Treat',
 'Teräsbetoni',
 'Powerglove',
 'Wishbone Ash',
 'God Dethroned',
 'Seth ',
 'Pink Cream 69',
 'Green Carnation',
 'The Exploited',
 'Nachtmystium',
 'Evildead',
 'Kagrra',
 'Deströyer 666',
 'Jorn',
 'In Mourning',
 'Goatmoon',
 'Lost Society',
 'Lost Soul ',
 'Mournful Congregation',
 'Skull Fist',
 'KMFDM',
 'Ravencult',
 'Virgin Black',
 'Ouroboros ',
 'I Declare War',
 'Watain',
 'Warcry ',
 'Metal Church',
 'Winds Of Plague',
 'Nirvana',
 'Issues',
 'Elffor',
 'Trauma ',
 'Impetigo',
 'Sadus',
 'The Cult',
 'Sixx:AM',
 'My Dying Bride',
 'Memphis May Fire',
 'Mägo De Oz',
 'Sabaton',
 'From Autumn To Ashes']

In [37]:

noms=[]
pays=[]
statuts=[]
genres=[]
annees=[]
paroles=[]
c=1
for elem in groupes13:
    print(elem)
    infos=scrapping_groupes(elem)
    print(infos)
    if infos!=elem:
        noms.append(infos[0])
        pays.append(infos[1])
        statuts.append(infos[2])
        genres.append(infos[3])
        annees.append(infos[4])
        try:
            paroles.append(infos[5])
        except IndexError:
            continue
    else:
        print("méthode spirit")
        infos2=scrapping_spirit(elem)
        print(infos2)
        noms.append(infos2[0])
        pays.append(infos2[1])
        genres.append(infos2[2])
        statuts.append(infos2[3])
        annees.append(infos2[4])
        paroles.append("NA")
    print(c,"/",len(groupes13),"groupes scrappés")
    c+=1

    

Eternal Tears Of Sorrow
Eternal Tears Of Sorrow
https://www.metal-archives.com/bands/Eternal_Tears_of_Sorrow/333
('Eternal Tears of Sorrow', 'Finland', 'Active', 'Melodic/Symphonic Death Metal', '1991-1992  , 1993  ..., 1993-1994   .., 1994-2003, 2004-', 'Autumn, Sorrow, Loss, Love, Death, Darkness')
1 / 50 groupes scrappés
Ministry
Ministry
https://www.metal-archives.com/bands/Ministry/59780
('Ministry', 'United States', 'Active', 'New Wave (early); Industrial/Electronic (mid); Industrial Rock/Metal (later)', '1981-2008, 2011-2013, 2014-', 'Love (early); Politics, Drugs, Religion, Humor, Antifascism (later)')
2 / 50 groupes scrappés
Sick Puppies
Sick Puppies
Sick Puppies  non présent dans Métal Archives
Sick Puppies
méthode spirit
('Sick Puppies', 'Australie', 'Metal Alternatif', 'Actif', '1997')
3 / 50 groupes scrappés
Quiet Riot
Quiet Riot
https://www.metal-archives.com/bands/Quiet_Riot/1709
('Quiet Riot', 'United States', 'Active', 'Hard Rock, Heavy Metal', '1973   1, 1973-1975   ,

Winds Of Plague
https://www.metal-archives.com/bands/Winds_of_Plague/42210
('Winds of Plague', 'United States', 'Active', 'Symphonic Deathcore', '2002-2005   , 2005-', 'Hatred, Murder, War, Domination')
37 / 50 groupes scrappés
Nirvana
Nirvana
https://www.metal-archives.com/bands/Nirvana_2002/11058
('Nirvana 2002', 'Sweden', 'Split-up', 'Death Metal', '1988   2002, 1988  , 1988-1992, 2007-2012', 'Metaphysical philosophy, Parapsychology')
38 / 50 groupes scrappés
Issues
Issues
Issues  non présent dans Métal Archives
Issues
méthode spirit
('Issues', 'Etats-Unis', 'Metalcore', 'Actif', '2012')
39 / 50 groupes scrappés
Elffor
Elffor
Elffor  non présent dans Métal Archives
Elffor
méthode spirit
('Elffor', 'Espagne', 'Black Epique', 'Actif', '1995')
40 / 50 groupes scrappés
Trauma 
Trauma 
https://www.metal-archives.com/bands/Trauma/3540456384
('Trauma', 'Mexico', 'Active', 'Thrash/Heavy Metal/Punk', '1993-', 'N/A')
41 / 50 groupes scrappés
Impetigo
Impetigo
https://www.metal-archives.com/ba

In [44]:
res={}
res['nom']=noms
res['pays']=pays
res['statut']=statuts
res['genre']=genres
res['years']=annees
res['lyrical themes']=paroles
df=pd.DataFrame.from_dict(res,orient="columns")
df

,nom,pays,statut,genre,years,lyrical themes
0,Eternal Tears of Sorrow,Finland,Active,Melodic/Symphonic Death Metal,"1991-1992 , 1993 ..., 1993-1994 .., 1994-2...","Autumn, Sorrow, Loss, Love, Death, Darkness"
1,Ministry,United States,Active,New Wave (early); Industrial/Electronic (mid);...,"1981-2008, 2011-2013, 2014-","Love (early); Politics, Drugs, Religion, Humor..."
2,Sick Puppies,Australie,Actif,Metal Alternatif,1997,NA
3,Quiet Riot,United States,Active,"Hard Rock, Heavy Metal","1973 1, 1973-1975 , 1975-2003, 2004-2008, ...","Life, Partying, Love, Metal, Society"
4,Theocracy,United States,Active,Epic Progressive Power Metal,2002-,"Christianity, Life, Theology"
5,Ljå,Norway,Active,Black Metal,"1992-2002 ø, 2002-","War, Evil, Sadness, Darkness"
6,Jag Panzer,United States,Active,Heavy/Power Metal,"1981 , 1981-1988, 1993-2011, 2013-","Destruction, Warfare, Literature, Shakespeare"
7,Upon This Dawning,Italie,Actif,Metalcore,2006,NA
8,TrickOrTreat,Germany,Active,Thrash/Groove Metal/Metalcore,1999-,N/A
9,Teräsbetoni,Finland,Active,Power Metal,"2003-2011, 2013, 2014, 2015-","Metal, Glory, Power, Justice, Honour, Battles"


In [39]:
df.to_csv('df13.csv', index=False)

## Sens Critique

In [4]:
driver = webdriver.Firefox()
driver.get("https://www.spirit-of-metal.com/liste_groupe.php?recherche_groupe=&lettre=&id_pays_recherche=0&id_style_recherche=0&dateCrea=0&nb_etoile=2")
scroll(driver,2)

time.sleep(3)

groupes=driver.find_elements_by_class_name("col-xs-9")
bands=[]
for e in groupes:
    e=e.text
    e=e.split("\n")[0]
    cleanr = re.compile('(\((.*)\))')
    e= re.sub(cleanr, '',e)
    bands.append(e)
print(len(bands))

1971


In [23]:
print(bands)

['12 Stones', '12012', '1349', '25 Ta Life', '3 Inches Of Blood', '30 Seconds To Mars', '36 Crazyfists', 'A Day To Remember', 'A Different Breed Of Killer', 'A Forest Of Stars', 'A Perfect Circle', 'A Plea For Purging', 'A Skylit Drive', 'A Static Lullaby', 'A Vain Attempt', 'A9', 'Aäkon Këëtrëh', 'Aaskereia', 'Abacabb', 'Abandon All Ships', 'Abbath', 'Abigail ', 'Abigail Williams', 'Abigor', 'Abingdon Boys School', 'Abnormality ', 'Abominable Putridity', 'Aborted', 'Aborym', 'Abruptum', 'Absu', 'Abysmal Dawn', 'Abysmal Torment', 'Abyssic Hate', 'Abyssos', 'AC-DC', 'Accept', 'Accuser', 'Ace Frehley', 'Acheron ', 'Acherontas', 'Acid Bath', 'Acid Drinkers', 'Acid King', 'Acid Witch', 'Acrania ', 'Ad Hominem', 'Adagio ', 'Adelitas Way', 'Adema', 'Adept', 'Adorned Brood', 'Adrenaline Mob', 'Adrian Von Ziegler', 'ADX', 'Aeon ', 'Aerosmith', 'Aes Dana', 'Aesma Daeva', 'Aeternam', 'Aeternus', 'AFI', 'After Forever', 'After The Burial', 'Agalloch', 'Agathocles', 'Agathodaimon', 'Agent Steel', 

In [40]:
bandes11=gps[:50]
bandes12=gps[50:100]
bandes13=gps[100:150]
bandes13

['Eternal Tears Of Sorrow',
 'Ministry',
 'Sick Puppies',
 'Quiet Riot',
 'Theocracy',
 'Ljå',
 'Jag Panzer',
 'Upon This Dawning',
 'Treat',
 'Teräsbetoni',
 'Powerglove',
 'Wishbone Ash',
 'God Dethroned',
 'Seth ',
 'Pink Cream 69',
 'Green Carnation',
 'The Exploited',
 'Nachtmystium',
 'Evildead',
 'Kagrra',
 'Deströyer 666',
 'Jorn',
 'In Mourning',
 'Goatmoon',
 'Lost Society',
 'Lost Soul ',
 'Mournful Congregation',
 'Skull Fist',
 'KMFDM',
 'Ravencult',
 'Virgin Black',
 'Ouroboros ',
 'I Declare War',
 'Watain',
 'Warcry ',
 'Metal Church',
 'Winds Of Plague',
 'Nirvana',
 'Issues',
 'Elffor',
 'Trauma ',
 'Impetigo',
 'Sadus',
 'The Cult',
 'Sixx:AM',
 'My Dying Bride',
 'Memphis May Fire',
 'Mägo De Oz',
 'Sabaton',
 'From Autumn To Ashes']

In [41]:

def alternative_spirit(elem):
    notes=[]
    driver = webdriver.Firefox()
    driver.get("https://www.senscritique.com/searchArtist?q={}".format(elem))
    elem=re.findall("[a-zA-Z]+",elem)
    elem=str(",".join(elem)).replace(","," ")
    a=driver.find_elements_by_css_selector("html body div#__next div.search__StyledGrid-sc-1mr42vn-0.deACLc div.search__StyledResultsColumn-sc-1mr42vn-4.igWMhV div.sk-hits.SearchArtist__HitsStyled-soe2rz-0.iFPMuN div.SearchArtistResult__Container-sc-19zp711-0.eACiIN a.SearchArtistResult__Title-sc-19zp711-5.bxizrx")
    lien=[i.get_attribute("href") for i in a]
    a1=lien[1]
    a1=a1+"#page-1/order-note/"
    driver.get(a1)
    favoris=driver.find_element_by_class_name("cvi-totallike").get_attribute("data-sc-like-count")
    b=driver.find_elements_by_class_name("erra-global")
    note=[]
    c=driver.find_elements_by_class_name("erra-ratings")
    for i in c:
        b=i.find_element_by_tag_name("a")
        if b.text!='-':
            note.append(float(b.text))
    moy=round(np.mean(notes),2)
    time.sleep(2)
    driver.quit()
    return elem,moy,favoris


In [80]:
test='Audrey Horne '
res=alternative_spirit(test)
res

('Audrey Horne', nan, '9')

In [42]:
def scrapping_sens_critique(elem):
    note=[]
    likes=driver.find_element_by_class_name("cvi-totallike").get_attribute("data-sc-like-count")
    c=driver.find_elements_by_class_name("erra-ratings")
    for i in c:
        b=i.find_element_by_tag_name("a")
        if b.text!='-':
            note.append(float(b.text))
    moy=round(np.mean(note),2)
    time.sleep(2)
    driver.quit()
    return elem,moy,likes



In [52]:
c=1
favoris=[]
moyennes=[]
groupes=[]
scrap2=[]
for elem in bandes13:
    try:
        driver = webdriver.Firefox()
        driver.get("https://www.senscritique.com/searchArtist?q={}".format(elem))
        a=driver.find_element_by_xpath("/html/body/div[1]/div[2]/div[3]/div[2]/div[1]/a").get_attribute("href")
        a=a+"#page-1/order-note/"
    except NoSuchElementException:
        print(elem," non trouvé.")
        scrap2.append(elem)
        print(elem," à rescrapper !")
    
    driver.get(a)
    if driver.current_url=="https://www.senscritique.com/#page-1/order-note/":
        driver.quit()
        alt=alternative_spirit(elem)
        favoris.append(alt[2])
        moyennes.append(alt[1])
        groupes.append(alt[0])
        print(alt[0],alt[1],alt[2])
    
        
    else:
        result=scrapping_sens_critique(elem)
        favoris.append(result[2])
        moyennes.append(result[1])
        groupes.append(result[0])
        print(result[0],result[1],result[2]) 
    print(c,"sur",len(bandes13),"groupes scrappés")
    c+=1


Eternal Tears Of Sorrow 6.67 4
1 sur 50 groupes scrappés
Ministry 7.7 55
2 sur 50 groupes scrappés
Sick Puppies 6.42 10
3 sur 50 groupes scrappés
Quiet Riot 5.97 6
4 sur 50 groupes scrappés
Theocracy 6.9 1
5 sur 50 groupes scrappés
Ljå 7.2 0
6 sur 50 groupes scrappés
Jag Panzer 7.05 0
7 sur 50 groupes scrappés


C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Upon This Dawning nan 0
8 sur 50 groupes scrappés
Treat 6.75 18
9 sur 50 groupes scrappés
Teräsbetoni nan 3
10 sur 50 groupes scrappés
Powerglove 7.2 6
11 sur 50 groupes scrappés
Wishbone Ash 7.17 33
12 sur 50 groupes scrappés
God Dethroned 6.5 2
13 sur 50 groupes scrappés
Seth  7.09 1143
14 sur 50 groupes scrappés
Pink Cream 69 nan 1
15 sur 50 groupes scrappés
Green Carnation 7.24 11
16 sur 50 groupes scrappés
The Exploited 7.08 28
17 sur 50 groupes scrappés
Nachtmystium 7.1 13
18 sur 50 groupes scrappés
Evildead 6.9 3
19 sur 50 groupes scrappés
Kagrra 7.0 5
20 sur 50 groupes scrappés
Deströyer 666 7.47 18
21 sur 50 groupes scrappés
Jorn 8.19 34
22 sur 50 groupes scrappés
In Mourning 7.58 9
23 sur 50 groupes scrappés
Goatmoon 7.38 2
24 sur 50 groupes scrappés
Lost Society 7.02 12
25 sur 50 groupes scrappés
Lost Soul  7.19 71
26 sur 50 groupes scrappés
Mournful Congregation 7.44 6
27 sur 50 groupes scrappés
Skull Fist 7.3 3
28 sur 50 groupes scrappés
KMFDM 7.07 29
29 sur 50 groupes scr

In [ ]:
#print(scrap2)

elem='Sixx:AM'
scrap2=[]
try:
    driver = webdriver.Firefox()
    driver.get("https://www.senscritique.com/searchArtist?q={}".format(elem))
    a=driver.find_element_by_xpath("/html/body/div[1]/div[2]/div[3]/div[2]/div[1]/a").get_attribute("href")
    a=a+"#page-1/order-note/"
except NoSuchElementException:
    print(elem," non trouvé.")
    scrap2.append(elem)
    print(elem," à rescrapper !")
    
driver.get(a)
if driver.current_url=="https://www.senscritique.com/#page-1/order-note/":
    driver.quit()
    alt=alternative_spirit(elem)
    favoris.append(alt[2])
    moyennes.append(alt[1])
    groupes.append(alt[0])
    print(alt[0],alt[1],alt[2])
    
        
else:
    result=scrapping_sens_critique(elem)
    favoris.append(result[2])
    moyennes.append(result[1])
    groupes.append(result[0])
    print(result[0],result[1],result[2])   

elem="Atrocity"
favoris=[]
moyennes=[]
groupes=[]
driver = webdriver.Firefox()
driver.get("https://www.senscritique.com/searchArtist?q={}".format(elem))
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

for elem in compte:
    b=elem.find_element_by_tag_name("span")
    print(b.text)

a=a+"#page-1/order-note/"
driver.get(a)

if driver.current_url=="https://www.senscritique.com/#page-1/order-note/":
    driver.quit()
    alt=alternative_spirit(elem)
    favoris.append(alt[2])
    moyennes.append(alt[1])
    groupes.append(alt[0])
    print(alt[0],alt[1],alt[2])


else:
    result=scrapping_sens_critique(elem)
    favoris.append(result[2])
    moyennes.append(result[1])
    groupes.append(result[0])
    print(result[0],result[1],result[2])
    print(c,"sur",len(bandes4),"groupes scrappés")

In [53]:
import pandas as pd
ratings={}
ratings["nom"]=groupes
ratings["note moyenne"]=moyennes
ratings["favoris"]=favoris
dfr=pd.DataFrame.from_dict(ratings,orient='columns')
dfr

,nom,note moyenne,favoris
0,Eternal Tears Of Sorrow,6.67,4
1,Ministry,7.70,55
2,Sick Puppies,6.42,10
3,Quiet Riot,5.97,6
4,Theocracy,6.90,1
5,Ljå,7.20,0
6,Jag Panzer,7.05,0
7,Upon This Dawning,NaN,0
8,Treat,6.75,18
9,Teräsbetoni,NaN,3


In [54]:

dfr["note moyenne"]=dfr["note moyenne"].fillna(0)
dfr=dfr.drop_duplicates()
dfr

,nom,note moyenne,favoris
0,Eternal Tears Of Sorrow,6.67,4
1,Ministry,7.70,55
2,Sick Puppies,6.42,10
3,Quiet Riot,5.97,6
4,Theocracy,6.90,1
5,Ljå,7.20,0
6,Jag Panzer,7.05,0
7,Upon This Dawning,0.00,0
8,Treat,6.75,18
9,Teräsbetoni,0.00,3


In [55]:
dfr.to_csv('ratings13.csv', index=False)

#Anomalie
url="https://www.senscritique.com/contact/Anomalie/2664988"
r = requests.get(url,headers=head)
print(url,r.status_code)
html_soup = BeautifulSoup(r.content, 'html.parser')
likes=html_soup.find_all('button')[2].text[:2]
notes=[]
for i in html_soup.find_all('a',{'class':'erra-global'}):
    try:
        b=float(i.text)
        notes.append(b)
    except ValueError:
        del i
    
moy=round(np.mean(notes),2)
print(moy,likes)

# On va recher l'identifant de la colonne
note_loc = dfr.columns.get_loc('note moyenne')
# On va cherche le n° d ela ligne 'b'
ligne_loc = dfr[dfr['band']=="Anomalie"].index
# On change la valeur
dfr.iloc[ligne_loc, note_loc] =7.47
dfr

In [56]:
#df=pd.read_csv("df4.csv")
#dfr=pd.read_csv("ratings4.csv")
res=pd.merge(df,dfr,on="nom")
res

,nom,pays,statut,genre,years,lyrical themes,note moyenne,favoris
0,Ministry,United States,Active,New Wave (early); Industrial/Electronic (mid);...,"1981-2008, 2011-2013, 2014-","Love (early); Politics, Drugs, Religion, Humor...",7.70,55
1,Sick Puppies,Australie,Actif,Metal Alternatif,1997,NA,6.42,10
2,Quiet Riot,United States,Active,"Hard Rock, Heavy Metal","1973 1, 1973-1975 , 1975-2003, 2004-2008, ...","Life, Partying, Love, Metal, Society",5.97,6
3,Theocracy,United States,Active,Epic Progressive Power Metal,2002-,"Christianity, Life, Theology",6.90,1
4,Ljå,Norway,Active,Black Metal,"1992-2002 ø, 2002-","War, Evil, Sadness, Darkness",7.20,0
5,Jag Panzer,United States,Active,Heavy/Power Metal,"1981 , 1981-1988, 1993-2011, 2013-","Destruction, Warfare, Literature, Shakespeare",7.05,0
6,Upon This Dawning,Italie,Actif,Metalcore,2006,NA,0.00,0
7,Teräsbetoni,Finland,Active,Power Metal,"2003-2011, 2013, 2014, 2015-","Metal, Glory, Power, Justice, Honour, Battles",0.00,3
8,Powerglove,Etats-Unis,Actif,Power Mélodique,2005,NA,7.20,6
9,Wishbone Ash,Royaume-Uni,Actif,Hard Rock,1969,NA,7.17,33


In [57]:
res.to_csv("final13.csv",index=False)

In [13]:
#res["lyrical themes"]=res["lyrical themes"].fillna("NA")
#res.to_csv("final4.csv",index=False)